<a href="https://colab.research.google.com/github/RaamRaam/tf2api/blob/master/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
print(str(tf.__version__))

In [ ]:
if tf.test.is_gpu_available():
  print(tf.test.gpu_device_name())
else:
  print('No GPU found')

In [ ]:
# !rm -rf tf2api
!git clone https://github.com/RaamRaam/tf2api.git

In [ ]:
import tf2api as tf2x
from tf2api.tfrecords import ds 

# tf2x=train()

In [ ]:
import os

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
class_names = ['zero','one','two','three','four','five','six','seven','eight','nine']

x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1)
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)

train_data={}
train_data['features']=x_train.astype(float)
train_data['lables']=y_train

test_data={}
test_data['features']=x_test.astype(float)
test_data['lables']=y_test

path=r'/content'

tf2x.SaveTFRecordSet(os.path.join(path, 'mnist_train.tfrecords'),train_data)
tf2x.SaveTFRecordSet(os.path.join(path, 'mnist_test.tfrecords'),test_data)
tf2x.create_classfile(os.path.join(path, 'classes.txt'),class_names)

In [ ]:
train_ds=ds()
tffilelist=!ls /content/mnist_train.tfrecords -ltrh
# tffilelist=[i.split(' \'')[-1][:-1] for i in tffilelist]
tffilelist=[i.split()[-1] for i in tffilelist]
train_ds.ReadTFRecordSet(tffilelist,10)

In [ ]:
test_ds=ds()
tffilelist=!ls /content/mnist_test.tfrecords -ltrh
# tffilelist=[i.split(' \'')[-1][:-1] for i in tffilelist]
tffilelist=[i.split()[-1] for i in tffilelist]
test_ds.ReadTFRecordSet(tffilelist,10)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
fig=plt.figure(figsize=(14,14))
plt.rcParams['figure.dpi'] = 250
plt.subplots_adjust(left=1, bottom=1, right=2, top=4)

select=test_ds.FilterTFRecordSet('lables',5)
number_of_images=10
ctr=1
for i in select.ds:
  if ctr<=number_of_images:
    # print(tf.reshape(i['features'],[28,28]).shape)
    fig.add_subplot(number_of_images, 5, ctr)
    plt.imshow(tf.reshape(i['features'],[28,28]))
    # print(i['lables'].numpy())
    ctr=ctr+1
  else:
    break

In [ ]:
def model():
  return tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.02),input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
  ])
# model().summary()
# keras.utils.plot_model(model(), 'my_first_model.png')

In [ ]:
a=tf2x.train()
a.name='first'
a.epochs=10
a.batch_size=512
a.lr_peak=2
a.lr_repeat=3
modes=['constant','stepup','stepdown','angledup','angleddown']
a.lr_mode=modes[0]
a.log_path='logs'
a.model=model
# a.lossfunction=tf.keras.losses.SparseCategoricalCrossentropy()
# a.optimizer=tf.keras.optimizers.SGD
a.call(train_ds,test_ds)




In [ ]:
%tensorboard --logdir logs

In [ ]:
a.batch_size=1024
a.call(train_ds,test_ds)

In [ ]:
!mkdir new
!mkdir new/first
a.save('new')

In [ ]:
b=tf2x.train()
b.name='first'
b.load('new')

In [ ]:
b.call(train_ds,test_ds)

In [ ]:
actuals, predictions=a.evaluate(train_ds.ds)
act_pred=[(i,actuals[i],predictions[i].index(max(predictions[i]))) for i in range(len(actuals))]
'Train accuray is ' + str(100*(1-len([act_pred[i] for i in range(len(act_pred)) if act_pred[i][1]!=act_pred[i][2]])/60000))


In [ ]:
actuals, predictions=a.evaluate(test_ds.ds)
act_pred=[(i,actuals[i],predictions[i].index(max(predictions[i]))) for i in range(len(actuals))]
'Test accuray is ' + str(100*(1-len([act_pred[i] for i in range(len(act_pred)) if act_pred[i][1]!=act_pred[i][2]])/10000))


In [ ]:
%tensorboard --logdir logs

In [ ]:
a.batch_size=1024
a.call(train_ds,test_ds)

In [ ]:
a=tf2x.train(hparams)
a.call()

In [ ]:
!mkdir first
a.save('first')

In [ ]:
model,hparams=a.load('first')

In [ ]:
def model():
  inputs = keras.Input(shape=(28,28,1), name='Inputs')
  x = layers.Conv2D(filters=32,kernel_size=(3,3), activation='relu', name='Conv_1')(inputs)
  x = layers.MaxPooling2D()(x)
  x = layers.Flatten()(x)
  x = layers.Dropout(0.1)(x)
  x = layers.Dense(64,activation='relu')(x)
  x = layers.BatchNormalization()(x)
  outputs = layers.Dense(10,activation='softmax')(x)
  return keras.Model(inputs=inputs, outputs=outputs)
model().summary()
keras.utils.plot_model(model(), 'my_first_model.png')

In [ ]:
def model():
  return tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.02),input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
  ])
model().summary()
keras.utils.plot_model(model(), 'my_first_model.png')

In [ ]:

def make_model():
  with tf.name_scope('Input'):
    inputs = keras.Input(shape=(28,28,1), name='Inputs')

  with tf.name_scope('Block1'):
    x = layers.Conv2D(filters=32,kernel_size=(3,3), activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.02),use_bias=False, name='B1_3x3')(inputs)
    x = layers.BatchNormalization(name='B1_BN')(x)
    x = layers.MaxPooling2D(name='B1_MP')(x)

  with tf.name_scope('Block2'):  
    x = layers.Conv2D(filters=64,kernel_size=(3,3), activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.02),use_bias=False, name='B2_3x3')(x)
    x = layers.BatchNormalization(name='B2_BN')(x)
    x = layers.MaxPooling2D(name='B2_MP')(x)

  with tf.name_scope('Block3'):  
    x = layers.Conv2D(filters=128,kernel_size=(3,3), activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.02),use_bias=False, name='B3_3x3')(x)
    x = layers.BatchNormalization(name='B3_BN')(x)

  with tf.name_scope('Output'):
    x = layers.Conv2D(filters=10,kernel_size=(3,3), activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.02),use_bias=False, name='B4_3x3')(x)
    x = layers.Flatten()(x)
    outputs = layers.Activation('softmax')(x)
  return keras.Model(inputs=inputs, outputs=outputs)
model().summary()
keras.utils.plot_model(model(), 'my_first_model.png', show_shapes=True)


In [ ]:
tf.saved_model.save(model,"/content/drive/My Drive/tf2apis/")

In [ ]:
!saved_model_cli show --dir "/content/drive/My Drive/tf2apis/" --tag_set serve --signature_def serving_default

In [ ]:
loaded = tf.saved_model.load("/content/drive/My Drive/tf2apis/")

In [ ]:
print(list(loaded.signatures.keys()))  # ["serving_default"]

In [ ]:
infer = loaded.signatures["serving_default"]

for i in test_ds['ds']:
    ftr=tf.reshape(i['features'],[1,28,28,1])
    pred=infer(tf.cast(ftr,tf.float16))
    pred_list=list(pred.values())[0].numpy()[0]
    print(pred_list)
    print('Actual:',i['lables'].numpy(), '\t Predicted:',tf.math.argmax(pred_list).numpy())
    plt.imshow(tf.reshape(i['features'],[28,28]))
    break